In [5]:
# Code to create a HuggingFace dataset from promoter and non-promoter FASTA files
import os
import random
from datasets import Dataset, DatasetDict
import pandas as pd
from typing import List, Dict

def parse_non_promoter_fasta(file_path: str) -> List[Dict]:
    """
    Parse non-promoter FASTA file with format:
    >DNA excision repair cross-complementing protein, gene_id:MYC6.7 [AB006707 AB020748 AB005233]
    GTTCATTTAGAATGTGTATCCACTATCCAGTGTTACTGGGATCTACAAATAACAAGTTTCC...
    """
    sequences = []
    current_header = ""
    current_seq = ""
    
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                # Save the previous sequence if it exists
                if current_header and current_seq:
                    # Extract gene ID if possible
                    gene_id = ""
                    if "gene_id:" in current_header:
                        try:
                            gene_id = current_header.split("gene_id:")[1].split()[0].strip()
                        except:
                            pass
                    
                    # Extract description
                    description = current_header
                    if "[" in description:
                        description = description.split("[")[0].strip()
                    
                    sequences.append({
                        'seq': current_seq,
                        'description': description,
                        'gene_id': gene_id,
                        'is_promoter': 0,  # 0 for non-promoter
                        'label': 0         # Label for classification
                    })
                
                # Start new sequence
                current_header = line[1:]  # Remove '>' character
                current_seq = ""
            else:
                current_seq += line
        
        # Add the last sequence
        if current_header and current_seq:
            gene_id = ""
            if "gene_id:" in current_header:
                try:
                    gene_id = current_header.split("gene_id:")[1].split()[0].strip()
                except:
                    pass
            
            description = current_header
            if "[" in description:
                description = description.split("[")[0].strip()
            
            sequences.append({
                'seq': current_seq,
                'description': description,
                'gene_id': gene_id,
                'is_promoter': 0,
                'label': 0
            })
    
    return sequences

def parse_promoter_fasta(file_path: str) -> List[Dict]:
    """
    Parse promoter FASTA file with format:
    >FP000001 AT1G01050_1 :+U EU:NC; range -200 to 50.
    CAAGTATCCTACATAGATTATAGGAGTGACCGCAAAAACACAAACTATGTTTCGTAATAA...
    """
    sequences = []
    current_header = ""
    current_seq = ""
    
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                # Save the previous sequence if it exists
                if current_header and current_seq:
                    parts = current_header.split()
                    fp_id = parts[0] if len(parts) > 0 else ""
                    gene_id = parts[1] if len(parts) > 1 else ""
                    range_info = ' '.join(parts[2:]) if len(parts) > 2 else ""
                    
                    sequences.append({
                        'seq': current_seq,
                        'fp_id': fp_id,
                        'gene_id': gene_id,
                        'range_info': range_info,
                        'is_promoter': 1,  # 1 for promoter
                        'label': 1         # Label for classification
                    })
                
                # Start new sequence
                current_header = line[1:]
                current_seq = ""
            else:
                current_seq += line
        
        # Add the last sequence
        if current_header and current_seq:
            parts = current_header.split()
            fp_id = parts[0] if len(parts) > 0 else ""
            gene_id = parts[1] if len(parts) > 1 else ""
            range_info = ' '.join(parts[2:]) if len(parts) > 2 else ""
            
            sequences.append({
                'seq': current_seq,
                'fp_id': fp_id, 
                'gene_id': gene_id,
                'range_info': range_info,
                'is_promoter': 1,
                'label': 1
            })
    
    return sequences

def create_dna_promoter_dataset(promoter_file: str, non_promoter_file: str, 
                               output_dir=None, train_split=0.8,
                               val_split=0.1, test_split=0.1, seed=42):
    """
    Create a balanced HuggingFace dataset from promoter and non-promoter files.
    
    Args:
        promoter_file: Path to the promoter FASTA file
        non_promoter_file: Path to the non-promoter FASTA file
        output_dir: Directory to save the dataset (optional)
        train_split, val_split, test_split: Dataset split proportions
        seed: Random seed for reproducibility
    """
    # Check that files exist
    if not os.path.exists(promoter_file):
        raise FileNotFoundError(f"Promoter file not found: {promoter_file}")
    if not os.path.exists(non_promoter_file):
        raise FileNotFoundError(f"Non-promoter file not found: {non_promoter_file}")
    
    # Parse files
    promoters = parse_promoter_fasta(promoter_file)
    non_promoters = parse_non_promoter_fasta(non_promoter_file)
    
    print(f"Found {len(promoters)} promoters and {len(non_promoters)} non-promoters")
    
    # Balance the dataset
    min_size = min(len(promoters), len(non_promoters))
    
    # Randomly sample to get balanced dataset
    random.seed(seed)
    sampled_promoters = random.sample(promoters, min_size)
    sampled_non_promoters = random.sample(non_promoters, min_size)
    
    # Combine data
    all_data = sampled_promoters + sampled_non_promoters
    random.shuffle(all_data)
    
    # Convert to DataFrame
    df = pd.DataFrame(all_data)
    
    # Split into train/val/test
    train_size = int(len(df) * train_split)
    val_size = int(len(df) * val_split)
    
    train_df = df[:train_size]
    val_df = df[train_size:train_size+val_size]
    test_df = df[train_size+val_size:]
    
    # Create HuggingFace dataset
    dataset_dict = DatasetDict({
        'train': Dataset.from_pandas(train_df),
        'validation': Dataset.from_pandas(val_df),
        'test': Dataset.from_pandas(test_df)
    })
    
    # Save dataset if output directory is provided
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        dataset_dict.save_to_disk(output_dir)
        print(f"Dataset saved to {output_dir}")
    
    return dataset_dict

# Example usage - replace these with your actual file paths
promoter_file = "/home/foresti/mdlm/CNNPromoterData/Arabidopsis_tata.fa"
non_promoter_file = "/home/foresti/mdlm/CNNPromoterData/Arabidopsis_non_prom_big.fa"

# Create and analyze the dataset
dataset = create_dna_promoter_dataset(
    promoter_file=promoter_file,
    non_promoter_file=non_promoter_file,
    output_dir="Arabidopsis_promoter_dataset_tata"
)

# Print dataset info
print(dataset)
print(f"Train: {len(dataset['train'])} examples")
print(f"Validation: {len(dataset['validation'])} examples")
print(f"Test: {len(dataset['test'])} examples")

# Check class balance
train_promoters = sum(dataset['train']['is_promoter'])
train_non_promoters = len(dataset['train']) - train_promoters
print(f"\nClass balance in training set:")
print(f"Promoters: {train_promoters} ({train_promoters/len(dataset['train'])*100:.1f}%)")
print(f"Non-promoters: {train_non_promoters} ({train_non_promoters/len(dataset['train'])*100:.1f}%)")

# Display a few examples
print("\nSample from training set:")
for i in range(min(3, len(dataset['train']))):
    example = dataset['train'][i]
    label = "Promoter" if example['is_promoter'] == 1 else "Non-Promoter"
    print(f"Example {i+1} ({label}):")
    print(f"Sequence: {example['seq'][:50]}... ({len(example['seq'])} bp)")
    print(f"Gene ID: {example.get('gene_id', 'N/A')}")
    print(f"Description: {example.get('description', 'N/A')}")
    print(f"Range Info: {example.get('range_info', 'N/A')}")
    print(f"Label: {example['label']}")
    print("-" * 80)
    print()

Found 1497 promoters and 11459 non-promoters


Saving the dataset (0/1 shards):   0%|          | 0/2395 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/299 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/300 [00:00<?, ? examples/s]

Dataset saved to Arabidopsis_promoter_dataset_tata
DatasetDict({
    train: Dataset({
        features: ['seq', 'description', 'gene_id', 'is_promoter', 'label', 'fp_id', 'range_info'],
        num_rows: 2395
    })
    validation: Dataset({
        features: ['seq', 'description', 'gene_id', 'is_promoter', 'label', 'fp_id', 'range_info'],
        num_rows: 299
    })
    test: Dataset({
        features: ['seq', 'description', 'gene_id', 'is_promoter', 'label', 'fp_id', 'range_info'],
        num_rows: 300
    })
})
Train: 2395 examples
Validation: 299 examples
Test: 300 examples

Class balance in training set:
Promoters: 1205 (50.3%)
Non-promoters: 1190 (49.7%)

Sample from training set:
Example 1 (Non-Promoter):
Sequence: TCAGTCCGGTGCTCTTCGAGTCTCACATGTATCAACACGAGATCAACTAG... (251 bp)
Gene ID: 
Description: retrotransposon like protein, AT4g16870, similarity to copia-like retrotransp
Range Info: None
Label: 0
---------------------------------------------------------------------------